In [2]:
import yfinance as yf
import pandas as pd
import datetime as dt
import requests as req
from bs4 import BeautifulSoup


In [ ]:
tickers = ["GNFC"]

In [9]:
def getFinancialYear():
    today = dt.datetime.today()
    current_year = today.year
    date_difference =  dt.date(today.year, 3, 31) - dt.date(today.year, today.month, today.day)

    if (date_difference <= 0):
        return current_year + 1
    else:
        return current_year

    

print(getFinancialYear())

-135 days, 0:00:00


In [19]:
# Get Cashflows
apv_api_key = '2N8UEPVRC2AWTK6B'

cashflows = req.get("https://www.alphavantage.co/query?function=CASH_FLOW&symbol=TCS&apikey=2N8UEPVRC2AWTK6B")
cashflows = cashflows.json()
annual_cf_df = pd.DataFrame(cashflows['annualReports']).sort_index(ascending=False)
dates = annual_cf_df['fiscalDateEnding'].values.tolist()
years = []

for date in dates:
    years.append(dt.datetime.strptime(date, "%Y-%m-%d").year)

In [57]:
fcf_df = pd.DataFrame()
fcf_df["Year"] = years
fcf_df["Operating Cashflow"] = annual_cf_df["operatingCashflow"].astype(int)
fcf_df["Cap Ex"] = annual_cf_df["capitalExpenditures"].astype(int)
fcf_df["FCF"] = fcf_df["Operating Cashflow"] - fcf_df["Cap Ex"]
growth_rate = [0]

for i in range(len(fcf_df["FCF"])):
    if i < len(fcf_df["FCF"]) - 1:
        pct_diff = (fcf_df["FCF"].iloc[i+1] - fcf_df["FCF"].iloc[i])/abs(fcf_df["FCF"].iloc[i])
        growth_rate.append(pct_diff * 100)

fcf_df["Growth Rate"]  = growth_rate
avg_growth = fcf_df["Growth Rate"].iloc[1:].mean()

avg_growth

-35.630076966509364